1. Importing and converting to Python Object

In [1]:
import json
import re
from functools import reduce

In [3]:
main_dir = '../JSON/'
f1 = open(main_dir + 'DidYouCatchTheBall_US.json')
f2 = open(main_dir + 'Fireballs20_US.json')
f3 = open(main_dir + 'Hackers_IN.json')
f4 = open(main_dir + 'Indianboys_IN.json')
f5 = open(main_dir + 'Indiangirls_IN.json')
f6 = open(main_dir + 'MathParkour_US.json')
f7 = open(main_dir + 'Raistar_IN.json')
f8 = open(main_dir + 'TangramsRace.json')
f9 = open(main_dir + 'TheDice_US.json')
data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)
data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)
data7 = json.load(f7)
data8 = json.load(f8)
data = json.load(f9)

In [4]:
new_string = json.dumps(data, indent = 2)
new_string1 = json.dumps(data, indent = 2)
# print(new_string)
print(new_string1)

{
  "gameId": "The dice",
  "teamCount": 4,
  "playersPerTeam": 3,
  "username": {
    "usernameId": ""
  },
  "visibility": true,
  "stateIdCount": 9,
  "transitionIdCount": 9,
  "connectionIdCount": 10,
  "dataLog": false,
  "states": [
    {
      "stateType": "START_STATE",
      "stateId": "74122bc887b34237fd75ac304b9c55bd_start",
      "game": "The dice",
      "positionX": 488.0,
      "positionY": 100.0,
      "inputConnections": [],
      "outputConnections": [
        "74122bc887b34237fd75ac304b9c55bd_connection_1"
      ],
      "globalVariables": []
    },
    {
      "stateType": "OUTPUT_STATE",
      "stateId": "74122bc887b34237fd75ac304b9c55bd_state_1",
      "game": "The dice",
      "positionX": 205.0,
      "positionY": 114.0,
      "inputConnections": [
        "74122bc887b34237fd75ac304b9c55bd_connection_1"
      ],
      "outputConnections": [
        "74122bc887b34237fd75ac304b9c55bd_connection_2"
      ],
      "description": "State",
      "displayText": {
     

In [5]:
for e in data["states"]:
    print(e["stateId"])


74122bc887b34237fd75ac304b9c55bd_start
74122bc887b34237fd75ac304b9c55bd_state_1
74122bc887b34237fd75ac304b9c55bd_state_2
74122bc887b34237fd75ac304b9c55bd_state_3
74122bc887b34237fd75ac304b9c55bd_state_4
74122bc887b34237fd75ac304b9c55bd_state_5
74122bc887b34237fd75ac304b9c55bd_state_6
74122bc887b34237fd75ac304b9c55bd_state_7
74122bc887b34237fd75ac304b9c55bd_state_8
74122bc887b34237fd75ac304b9c55bd_state_9


In [7]:
print(len(data["states"]))

10


2. Defining Graph

In [8]:
class Graph:
    global adj, cnt
    def __init__(self, V): #this part like a constructor...
        self.V = V
        self.adj = [[] for i in range(V)]
        self.cnt = 0

    def add_edge(self, source, dest):
        self.adj[source].append(dest)
        
    def is_cyclic_util(self, i, visited, rec_stack):
        if(rec_stack[i]):
            self.cnt += 1
            return
        if(visited[i]): return
        visited[i] = True
        rec_stack[i] = True
        children = self.adj[i]
        
        for c in children:
            self.is_cyclic_util(c, visited, rec_stack)
        
        rec_stack[i] = False
        return
    
    def count(self):
        visited = [False for i in range(self.V)]
        rec_stack = [False for i in range(self.V)]
        for i in range(self.V):
            self.is_cyclic_util(i, visited, rec_stack)
        return self.cnt

3. Methods for counting loops through states and connections

In [9]:
#this method hashing state to its corresponding index
#Important: if the number of state does not
def index(s, dic, idx):
    tmp = re.findall("\d+\Z", s)
    return 0 if len(tmp) == 0 else int(tmp[0])
# print(index("74122bc887b34237fd75ac304b9c55bd_state_10"))

In [10]:
#this method create a dictonary: connection = (state_from, state_to)
#return type: dictionary
def con_to_state(lst):
    dic = {}
    idx = 0
    state_to_idx = {}
    for e in lst:
        if(state_to_idx.get(e["connectionFrom"]) == None):
            state_to_idx[e["connectionFrom"]] = idx
            idx += 1
        if(state_to_idx.get(e["connectionTo"]) == None):
            state_to_idx[e["connectionTo"]] = idx
            idx += 1
        dic[e["connectionId"]] = (state_to_idx[e["connectionFrom"]], state_to_idx[e["connectionTo"]])
    return dic


In [11]:
def create_graph(data):
    graph = Graph(len(data["states"]))
        
    conv = con_to_state(data["connections"])
    
    for e in data["states"]:
        # print(e["outputConnections"])
        for oc in e["outputConnections"]:
            start = conv[oc][0]
            end = conv[oc][1]
            graph.add_edge(start, end)
    # print(graph.adj)
    return graph


In [12]:
def count_loop(graph):
    return graph.count()

def count_if_else(graph):
    return reduce(lambda acc, cur: acc + len(cur) if len(cur) > 1 else acc, graph.adj, 0)

4. Implementing: counting loops and count if-else statements

In [13]:

graph = create_graph(data)
print(count_loop(graph))
print(count_if_else(graph))


0
6
